In [13]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from pybliometrics.scopus import AbstractRetrieval
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime
import json
import pandas as pd
import time
import re
import requests

# Scopus Search API
https://dev.elsevier.com/documentation/SCOPUSSearchAPI.wadl

In [14]:
base_url = 'http://api.elsevier.com/content/search/scopus?'

# search in title, abstract, and key
scope = 'TITLE-ABS-KEY'

# formulating the query structure
term = '(engineering)' 
# terms1 = '({python})'
# terms2 = '({machine learning} OR {big data} OR {artificial intelligence})'
# terms = '({} AND {})'.format(terms1, terms2)

# insert your personal key (it is free and available on https://dev.elsevier.com/)
# key = '376c53d75af00e4f467ca8eea0f0dffb'
key = '72eb9f8a575aacdd4d8e1a982b635a6c'
apiKey = f'&apiKey={key}'

# period
year = 2017
date = f'&date={year}'

# it is the maximum number of results per query for a free account
count = '&count=25'
sort = '&sort=coverDate'
view = '&view=standard'
cluster = '&cluster=scosrctype%2C"j"%2Ct%2C"p"%2Ct%2Bscolang%2C"English"%2Ct%2Bscosubtype%2C"ar"%2Ct%2C"cp"%2Ct%2Bscofreetoread%2C%22repository%22%2Ct'

# list of all subjects in Scopus database
subjects = 'ENGI'

# geopy
geolocator = Nominatim(user_agent='cj')

## Old Template

In [3]:
# this function sends a request and returns the total articles, the #starting position of the first article, and the metadata of each #article.
def search_scopus(url):
    res = requests.get(url)
    if res.status_code == 200:
        content = json.loads(res.content)['search-results']
        total = content['opensearch:totalResults']
        start = content['opensearch:startIndex']
        metadata = content['entry']
        return int(total), int(start), metadata
    else:
        error = json.loads(res.content)['service-error']['status']
        print(res.status_code, error['statusText'])

In [ ]:
# starting index of the results for display
# starting index refers to number position of not pages
start_index = 0

while True:
    start = '&start={}'.format(start_index)
    subj = '&subj={}'.format(subjects)
    query = 'query=' + scope + term + date + start + count + sort + subj + apiKey + view + cluster
    url = base_url + query
    # total results per subject, starting index of first result in
    # each query and data
    total, start_index, data = search_scopus(url)
    if start_index == 0:
        metadata = pd.DataFrame(data)
    else:
        metadata = pd.concat([metadata, pd.DataFrame(data)], ignore_index=True)
    print(start_index)
    # save data now in SQL (not shown here)
    # check how many results need to be retrieved
    remain = total - start_index - len(data)
    print(len(metadata))
    print(remain)
    if len(metadata) >= 5000:
        break # breaking from while loop
    elif remain > 0:
        start_index += 25 # to search next 25 results

### Starting index more than 5,000

In [ ]:
# # starting index of the results for display
# # starting index refers to number position of not pages
# start_index = 5000

# while True:
#     start = '&start={}'.format(start_index)
#     subj = '&subj={}'.format(subjects)
#     query = 'query=' + scope + term + date + start + count + sort + subj + apiKey + view + cluster
#     url = base_url + query
#     # total results per subject, starting index of first result in
#     # each query and data
#     total, start_index, data = search_scopus(url)
#     if start_index == 0:
#         pass
#         # metadata = pd.DataFrame(data)
#     else:
#         metadata = pd.concat([metadata, pd.DataFrame(data)], ignore_index=True)
#     print(start_index)
#     # save data now in SQL (not shown here)
#     # check how many results need to be retrieved
#     remain = total - start_index - len(data)
#     print(len(metadata))
#     print(remain)
#     if remain > 0:
#         start_index += 25 # to search next 25 results
#     else:
#         break # breaking from while loop

In [ ]:
df = metadata
display(df)

In [47]:
df.to_csv(f'scopus_2017.csv', index=False)


## New Template

In [6]:
# starting index of the results for display
# starting index refers to number position of not pages
start_index = 0

scopus_id = []
while True:
    start = '&start={}'.format(start_index)
    subj = '&subj={}'.format(subjects)
    query = 'query=' + scope + term + date + start + count + sort + subj + apiKey + view + cluster
    url = base_url + query

    # total, start_index, data = search_scopus(url)

    res = requests.get(url)
    results = res.json()
    total = int(results['search-results']['opensearch:totalResults'])
    start_index = int(results['search-results']['opensearch:startIndex'])
    for r in results['search-results']['entry']:
        scopus_id.append(str(r['dc:identifier']))
    print(start_index)
    # save data now in SQL (not shown here)
    # check how many results need to be retrieved
    remain = total - start_index - len(scopus_id)
    print(len(scopus_id))
    print(remain)
    # if remain > 0:
    #     start_index += 25 # to search next 25 results
    if len(scopus_id) >= 5000:
        break # breaking from while loop
    elif remain > 0:
        start_index += 25 # to search next 25 results

0
25
56415
25
50
56365
50
75
56315
75
100
56265
100
125
56215
125
150
56165
150
175
56115
175
200
56065
200
225
56015
225
250
55965
250
275
55915
275
300
55865
300
325
55815
325
350
55765
350
375
55715
375
400
55665
400
425
55615
425
450
55565
450
475
55515
475
500
55465
500
525
55415
525
550
55365
550
575
55315
575
600
55265
600
625
55215
625
650
55165
650
675
55115
675
700
55065
700
725
55015
725
750
54965
750
775
54915
775
800
54865
800
825
54815
825
850
54765
850
875
54715
875
900
54665
900
925
54615
925
950
54565
950
975
54515
975
1000
54465
1000
1025
54415
1025
1050
54365
1050
1075
54315
1075
1100
54265
1100
1125
54215
1125
1150
54165
1150
1175
54115
1175
1200
54065
1200
1225
54015
1225
1250
53965
1250
1275
53915
1275
1300
53865
1300
1325
53815
1325
1350
53765
1350
1375
53715
1375
1400
53665
1400
1425
53615
1425
1450
53565
1450
1475
53515
1475
1500
53465
1500
1525
53415
1525
1550
53365
1550
1575
53315
1575
1600
53265
1600
1625
53215
1625
1650
53165
1650
1675
53115
1675
1700
53065

In [15]:
dataset = []
for id in scopus_id:
    url = ('http://api.elsevier.com/content/abstract/scopus_id/' + id)
    # url = (
    #         'http://api.elsevier.com/content/abstract/scopus_id/'
    #         + id
    #         + '?field=authors,title,publicationName,volume,issueIdentifier,'
    #         + 'prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType'
    #     )
    resp = requests.get(
            url,
            headers={
                'Accept':'application/json',
                'X-ELS-APIKey': key
            }
        )
    data = json.loads(resp.text.encode('utf-8'))
    try:
        title = data['abstracts-retrieval-response']['coredata']['dc:title']
    except:
        title = ''
    try:
        abstracts = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
    except:
        abstracts = ''
    try:
        description = data['abstracts-retrieval-response']['coredata']['dc:description']
    except:
        description = ''
    try:
        doi = 'doi:' + data['abstracts-retrieval-response']['coredata']['prism:doi']
        ab = AbstractRetrieval(data['abstracts-retrieval-response']['coredata']['prism:doi'])
        authors = ', '.join([au.indexed_name for au in ab.authors])
    except:
        doi = ''
        try:
            authors = ', '.join([au['ce:indexed-name'] for au in data['abstracts-retrieval-response']['coredata']['dc:creator']['author']])
        except:
            authors = ''
    try:
        affiliation = data['abstracts-retrieval-response']['affiliation'][0]['affilname']
    except:
        affiliation = ''
    try:
        city = data['abstracts-retrieval-response']['affiliation'][0]['affiliation-city']
        location = geolocator.geocode(city)
        latitude = location.latitude
        longitude = location.longitude
    except:
        city = ''
        latitude = ''
        longitude = ''
    try:
        country = data['abstracts-retrieval-response']['affiliation'][0]['affiliation-country']
    except:
        country = ''

    seq = {
        'authors': authors,
        'title': title,
        'abstracts': abstracts,
        'description': description,
        'doi': doi,
        'affiliation': affiliation,
        'city': city,
        'country': country,
        'latitude': latitude,
        'longitude': longitude
    }
    dataset.append(seq)

In [16]:
dataset

[{'authors': 'Xu T.',
  'title': 'Pollution Withstand Voltage Characteristics of Y-shape Insulator String on UHVAC Transmission Lines',
  'abstracts': "© 2017, High Voltage Engineering Editorial Department of CEPRI. All right reserved.The Y-shape insulator string for UHVAC transmission line can realize optimization of wire suspension modes and insulator string performance by changing the string structure, which is of great practical values for special transmission corridors and bad weather conditions with excessively high costs of cleaning. Therefore, voltage-endurance tests and electric field simulation calculations are conducted against UHV Y-shape, V-shape, and I-shape insulator strings, and recommended a new type of Y-shape insulator string structure which is applicable to China's UHVAC transmission lines and insulation design.Test results indicate that, firstly,the relationship between U50% and salt deposition density is nonlinear and negatively exponentially decreases when the SD

In [17]:
df = pd.DataFrame(dataset)
df.to_csv(f'data_{year}.csv', sep='|', index=False)

# Source
scopus api - https://towardsdatascience.com/title-using-python-for-everyday-tasks-an-example-of-retrieving-metadata-from-scopus-database-4339610a86f2

scrape google scholar- https://medium.com/@nandinisaini021/scraping-publications-of-aerial-image-research-papers-on-google-scholar-using-python-a0dee9744728